# Chat with pdf file

Langchain 是一個讓使用者可以與其 PDF 檔案進行對話的工具。這意味著使用者可以以對話格式向 PDF 檔案提問，並獲得回答。例如，使用者可以問 PDF 檔案「法國的首都是哪裡？」，PDF 檔案會回答「巴黎」。Langchain 可以用於任何 PDF 檔案，是一個讓使用者更充分利用 PDF 檔案的好方法。

## 套件安裝與環境設置

### 安裝套件包



In [ ]:
!pip install openai
!pip install langchain
!pip install PyPDF2
!pip install chromadb
!pip install tiktoken
!pip install pymupdf
!pip install pypdf

### 環境參數設置

本章節中，將會使用到 OpenAI API key。

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "sk-xxxx" # 會是 sk-XXXX 樣式的字樣

### 連接至 Google Drive

In [ ]:
# connect your Google Drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"
file_path = '/content/gdrive/My Drive/field-guide-to-data-science.pdf'

## 讀取 PDF 檔案的多種方法
這邊將介紹多少可以讀取 PDF 的方式

* PyPdf2
* PyPdfLoader
* PyMuPDFLoader

官方連結：[Document_loaders](https://python.langchain.com/en/latest/modules/indexes/document_loaders/examples/pdf.html#pdf)



### 讀取 PDF 檔案 - PyPDF2 作法


In [ ]:
# 從 PyPDF2 引入 PdfReader class 進行作用
from PyPDF2 import PdfReader

# 擺放 Pdf 的檔案路徑，這邊是採用 Google Drive 路徑做示範：'/content/gdrive/MyDrive/field-guide-to-data-science.pdf'。
pdfreader = PdfReader(file_path)

# 從路徑中讀取檔案，並且放進名為 raw_text 變數當中做存放
raw_text = ''
for i, page in enumerate(pdfreader.pages):
    text = page.extract_text()
    if text:
        raw_text += text

In [ ]:
# 顯示部分文字出來瞅瞅
raw_text[:100]

### 讀取 PDF 檔案 - PyPdfLoader

用 pypdf 將 PDF 載入成包含頁面內容和頁碼元資料的文件陣列。每個文件都包含了該頁面的內容和元資料，例如頁碼等。

參考用官方連結：[Using PyPDF](https://python.langchain.com/en/latest/modules/indexes/document_loaders/examples/pdf.html#using-pypdf)



In [ ]:
from langchain.document_loaders import PyPDFLoader
pypdf_loader = PyPDFLoader(file_path)
pypdf_doc = pypdf_loader.load()

In [ ]:
pypdf_doc[20]

### 讀取 PDF 檔案 - PyMuPDFLoader

這是 PDF 解析選項中最快的，並且包含有關 PDF 及其頁面的詳細元資料，以期高渲染速度聞名。

參考用官方連結：[Using PyMuPDFLoader](https://python.langchain.com/en/latest/modules/indexes/document_loaders/examples/pdf.html#PyMuPDFLoader)



In [ ]:
from langchain.document_loaders import PyMuPDFLoader
pyMuPDFLoader = PyMuPDFLoader(file_path)
pyMuPDF_doc = pyMuPDFLoader.load()

In [ ]:
pyMuPDF_doc[2]

## 使用 QA_Chain 與文件對話！

Langchain中的 load_qa_chain 函數用於加載一個可以用來回答問題的 Chain。 chain_type 參數可用於指定加載的鏈的類型。 chain_type 的可能值為 stuff, map_reduce, refine, map_rerank。

* Stuff: 把所有的文本一次性傳給 LLM 進行總結。如果文本長度超過 LLM Token 時將會炸裂，對長文不會使用這個方式。（一般來說都不會用就是）
* map_reduce: 此法是先將文本分成多個小 batch 後，並針對每個小 batch 進行總結。
* refine: 此法將文本分成多個小 Batch 之後，有順序的先對第一個 batch 總結，之後結合第二個 batch 進行總結，以此類推，可以增加上下文連貫性。
* map_rerank: 此法比較像是 Retrival ，會將文本與提出的問題進行比對，找到最接近的那一項，接著交給 LLM，在接手 LLM 的回答。


官方連結：[Question Answering](https://python.langchain.com/en/latest/modules/chains/index_examples/question_answering.html)


In [ ]:
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain

# 讀取文件
pyMuPDFLoader = PyMuPDFLoader(file_path)
pyMuPDF_doc = pyMuPDFLoader.load()


In [ ]:
# 建構 QA_chain, Chain_type 有 stuff, map_reduce, refine, map_rerank
chain = load_qa_chain(llm=OpenAI(), chain_type="stuff") #當你出現 error 4097 token 錯誤資訊時，將 stuff 改成 map_reduce

# 問問題時間！
query = "what is the main point in this book?"
chain.run(input_documents=pyMuPDF_doc, question=query)

## 切割文件機制

Langchain 中的 Text Splitters 是一種工具，可用於將長文本拆分為更小、更易於管理的塊。這可用於各種任務，例如：

*   總結文本：通過將文本分成較小的塊，可以更容易地識別文本的要點並以簡潔的方式進行總結。
*   索引文本：通過將文本拆分成更小的塊，可以更輕鬆地索引文本，以便更有效地搜索它。

Langchain 中的 Text Splitters 是一個強大的工具，可以通過多種方式對文本進行處理和分析。如果您正在處理大段文本，那麼 Text Splitters 可能是你正在尋找的。


官方連結：[Text Splitters](https://python.langchain.com/en/latest/modules/indexes/text_splitters/getting_started.html#getting-started)

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

In [ ]:
pages = pyMuPDFLoader.load_and_split(splitter)

In [ ]:
pages[0]

In [ ]:
pages[1]

## 向量化小文本並放進向量資料庫中


### 讀取 Embedding

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings

In [ ]:
# Download embeddings from OpenAI
embeddings = OpenAIEmbeddings()

### 存放至 Chroma - 向量資料庫

官方連結：[Chroma](https://python.langchain.com/en/latest/modules/indexes/vectorstores/examples/chroma.html#chroma)

In [ ]:
from langchain.vectorstores import Chroma

In [ ]:
db = Chroma.from_documents(documents=pages, embedding=embeddings, persist_directory='db')

In [ ]:
db

## 多種使用 Chain 與文件對話的方式

* RetrievalQA
* ConversationalRetrievalChain



### RetrievalQA

RetrievalQA Chain 用意是在通過檢索向量資料庫中與問題最相近的向量進行回答，當中我們引入新的觀念 Retriever，透過 Retriever 來進行比較相似度，方法有兩個，分別為'mmr' 與 'similarity'，還可以設置要比較的數量

* Chain 官方連結：[Retrieval Question/Answering](https://python.langchain.com/en/latest/modules/chains/index_examples/vector_db_qa.html)
* VectorStore Retriever 官方連結：[VectorStore Retriever](https://python.langchain.com/en/latest/modules/indexes/retrievers/examples/vectorstore-retriever.html#vectorstore-retriever)


In [ ]:
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

# 讀取文件
pyMuPDFLoader = PyMuPDFLoader(file_path)
pyMuPDF_doc = pyMuPDFLoader.load()

# 切割文件
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
#pages = pyMuPDFLoader.load_and_split(splitter)
pages = splitter.split_documents(pyMuPDF_doc)

# 選擇要用的 embedding 並存放進向量資料庫
embeddings = OpenAIEmbeddings()
db = Chroma.from_documents(pages, embeddings)

# 使用 retrievaler 進行檢索
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k":5})

# 建構 QA Chain 來進行問答
qa = RetrievalQA.from_chain_type(
    llm=OpenAI(), chain_type="stuff", retriever=retriever, return_source_documents=True)


In [ ]:
query = "what is the main point in this book?"
result = qa({"query": query})

In [ ]:
result["result"]

In [ ]:
result["source_documents"]

### ConversationalRetrievalChain

官方連結：[Chat Over Documents with Chat History](https://python.langchain.com/en/latest/modules/chains/index_examples/chat_vector_db.html#chat-over-documents-with-chat-history)

In [ ]:
from langchain.llms import OpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.memory import ConversationBufferMemory


# 讀取文件
pyMuPDFLoader = PyMuPDFLoader(file_path)
pyMuPDF_doc = pyMuPDFLoader.load()

# 切割文件
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
#pages = pyMuPDFLoader.load_and_split(splitter)
pages = splitter.split_documents(pyMuPDF_doc)

# 選擇要用的 embedding 並存放進向量資料庫
embeddings = OpenAIEmbeddings()
db = Chroma.from_documents(pages, embeddings)

# 使用 retrievaler 進行檢索
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k":5})

# 建構 Memorry 來保存聊天記錄
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# 建構 QA Chain 來進行問答
qa = ConversationalRetrievalChain.from_llm(
    llm=OpenAI(), chain_type="stuff", retriever=retriever, memory=memory)


In [ ]:
query = "what is the main point in this book?"
result = qa({"question": query})

In [ ]:
result["answer"]

In [ ]:
query = "can you rewrite it?"
result = qa({"question": query})

In [ ]:
result["answer"]